In [4]:
import pandas as pd
import geopandas as gpd
import folium
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import pickle

In [1]:
import yaml
config_path = '/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
nutzungsdaten_path = config['data']['nutzungsdaten']

### Read in Data

In [6]:
with open(nutzungsdaten_path + 'nutzungsdaten_complete.pkl', 'rb') as handle:
    df = pickle.load(handle)

### Check Data

In [7]:
df.nutzart.unique()

array(['Bahnverkehr', 'Fläche besonderer funktionaler Prägung',
       'Fließgewässer', 'Flugverkehr', 'Fläche gemischter Nutzung',
       'Friedhof', 'Gehölz', 'Industrie- und Gewerbefläche',
       'Landwirtschaft', 'Platz', 'Stehendes Gewässer',
       'Sport-, Freizeit- und Erholungsfläche', 'Straßenverkehr', 'Sumpf',
       'Tagebau, Grube, Steinbruch', 'Unland/Vegetationslose Fläche',
       'Wald', 'Weg', 'Wohnbaufläche', 'Moor', 'Halde', 'Heide',
       'Hafenbecken', 'Schiffsverkehr'], dtype=object)

In [8]:
df.bez.nunique()

66

### Relabel to our use case

In [9]:
df['Label'] = ''
df['Label'] = np.where(df.nutzart.isin(['Fließgewässer', 'Stehendes Gewässer', 'Hafenbecken', 'Schiffsverkehr']), 'Wasser', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Wald', 'Forstwirtschaft']), 'Wald', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Wohnbaufläche']), 'Wohnbaufläche', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Straßenverkehr', 'Weg']), 'Verkehrsfläche/Weg', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Landwirtschaft']), 'Landwirtschaft', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Industrie- und Gewerbefläche']), 'Gewerbe', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Bahnverkehr', 'Flugverkehr']), 'Bahn/Flughafen', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Sport-, Freizeit- und Erholungsfläche']), 'Sport/Freizeit', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Fläche besonderer funktionaler Prägung']), 'Fläche besonderer funktionaler Prägung', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Fläche gemischter Nutzung']), 'Fläche gemischter Nutzung', df['Label'])

#### Subset only to Inner City for Plotting

In [10]:
from shapely.geometry import box
bbox = box(*[11.483459,48.056972,11.704559,48.196303])
df_subset = df[df.geometry_4326.intersects(bbox)]

### Plot some data

In [11]:
m = folium.Map(location  = [df.iloc[0,:].geometry_4326.centroid.y, df.iloc[0,:].geometry_4326.centroid.x], zoom_start = 12)
style_options = {
    'fillColor': '#0000ff',      # Fill color in hexadecimal format
    'color': '#0000ff',          # Line color in hexadecimal format
    'weight': 2,                 # Line weight in pixels
    'fillOpacity': 0.2           # Fill opacity (0.0 transparent to 1.0 opaque)
}
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True).add_to(m)

for label in tqdm(df_subset.Label.unique(), leave = False):
    folium.GeoJson(df_subset[df_subset.Label == label].geometry_4326, style_function=lambda x:style_options, name = label).add_to(m)
#folium.GeoJson(df[df.nutzart == 'Wald'].geometry_4326, style_function=lambda x:style_options, name = 'Water').add_to(m)
folium.LayerControl().add_to(m)
#m
m.save("Nutzungsdaten_Exploration.html")

  0%|          | 0/11 [00:00<?, ?it/s]